## 0. 概述
- 造成线程安全问题的主要诱因有两点，一是存在共享数据(也称临界资源)，二是存在多条线程共同操作共享数据。
- 可能需要这样一个方案，当存在多个线程操作共享数据时，需要保证同一时刻有且只有一个线程在操作共享数据，其他线程必须等到该线程处理完数据后再进行，这种方式有个高尚的名称叫互斥锁

---

## 1. synchronized 的三种应用方式
- 修饰实例方法，作用于当前实例加锁，进入同步代码前要获得当前实例的锁
- 修饰静态方法，作用于当前类对象加锁，进入同步代码前要获得当前类对象的锁
- 修饰代码块，指定加锁对象，对给定对象加锁，进入同步代码库前要获得给定对象的锁

---

### 1.1 synchronized 作用于实例方法
- 所谓的实例对象锁就是用 synchronized 修饰实例对象中的实例方法，注意是实例方法不包括静态方法

```
public class AccountingSync implements Runnable {
    // 共享资源(临界资源)
    static int i = 0;

    /**
     * synchronized 修饰实例方法
     */
    public synchronized void increase() {
        i++;
    }
    
    @Override
    public void run() {
        for (int j = 0; j < 1000000; j++) {
            increase();
        }
    }
    
    public static void main(String[] args) throws InterruptedException {
        AccountingSync instance = new AccountingSync();
        
        // 注意两个线程用的是同一个示例
        Thread t1 = new Thread(instance);
        Thread t2 = new Thread(instance);
        t1.start();
        t2.start();
        t1.join();
        t2.join();
        
        // 输出结果: 2000000
        System.out.println(i);
    }
}
```

---

### 1.2 synchronized 作用于静态方法
- 当 synchronized 作用于静态方法时，其锁就是当前类的 class 对象锁。由于静态成员不专属于任何一个实例对象，是类成员，因此通过 class 对象锁可以控制静态 成员的并发操作。
- 需要注意的是如果一个线程 A 调用一个实例对象的非 static synchronized 方法，而线程 B 需要调用这个实例对象所属类的静态 synchronized 方法，是允许的，不会发生互斥现象。

```
public class AccountingSyncClass implements Runnable {
    static int i = 0;

    /**
     * 作用于静态方法, 锁是当前 class 对象,也就是
     * AccountingSyncClass 类对应的 class 对象
     */
    public static synchronized void increase() {
        i++;
    }

    /**
     * 非静态, 访问时锁不一样不会发生互斥
     */
    public synchronized void increase4Obj() {
        i++;
    }

    @Override
    public void run() {
        for (int j = 0; j < 1000000; j++) {
            increase();
        }
    }
    
    public static void main(String[] args) throws InterruptedException {
        // new 新实例
        Thread t1 = new Thread(new AccountingSyncClass());
        
        // new 新实例
        Thread t2 = new Thread(new AccountingSyncClass());
        
        // 启动线程
        t1.start();
        t2.start();

        t1.join();
        t2.join();
        
        // 虽然是两个实例，但是调用的是 static synchronized 方法，共享一个 class 锁
        // 输出结果: 2000000
        System.out.println(i);
    }
}
```

---

### 1.3 synchronized 同步代码块
- 将 synchronized 作用于一个给定的实例对象 instance，每次当线程进入 synchronized 包裹的代码块时就会要求当前线程持有 instance 实例对象锁。这样也就保证了每次只有一个线程执行 i++; 操作

```
public class AccountingSync implements Runnable{
    static AccountingSync instance = new AccountingSync();
    static int i = 0;
    
    @Override
    public void run() {
        // 省略其他耗时操作....
        
        // 使用同步代码块对变量 i 进行同步操作, 锁对象为 instance
        synchronized(instance) {
            for (int j = 0; j < 1000000; j++) {
                i++;
            }
        }
    }
    
    public static void main(String[] args) throws InterruptedException {
        Thread t1 = new Thread(instance);
        Thread t2 = new Thread(instance);
        
        t1.start();
        t2.start();
        
        t1.join();
        t2.join();
        
        System.out.println(i);
    }
}
```

- 可以使用 this 对象(代表当前实例)或者当前类的 class 对象作为锁

```
// this, 当前实例对象锁
synchronized(this){
    for(int j = 0; j < 1000000; j++) {
        i++;
    }
}

// class 对象锁
synchronized(AccountingSync.class) {
    for(int j = 0; j < 1000000; j++) {
        i++;
    }
}
```

---

## 2. synchronized 底层语义原理
- Java 虚拟机中的同步(Synchronization) 基于进入和退出管程(Monitor)对象实现，无论是显式同步(有明确的 monitorenter 和 monitorexit 指令, 即同步代码块)还是隐式同步
- 同步方法并不是由 monitorenter 和 monitorexit 指令来实现同步的，而是由方法调用指令读取运行时常量池中方法的 ACC_SYNCHRONIZED 标志来隐式实现的

---

### 2.1 理解 Java 对象头与 Monitor
- 在 JVM 中，对象在内存中的布局分为三块区域：对象头、实例数据和对齐填充
  - 实例变量：存放类的属性数据信息，包括父类的属性信息，如果是数组的实例部分还包括数组的长度，这部分内存按4字节对齐
  - 填充数据：由于虚拟机要求对象起始地址必须是8字节的整数倍。填充数据不是必须存在的，仅仅是为了字节对齐，这点了解即可

- 顶部，是 Java 头对象，它实现 synchronized 的锁对象的基础
- synchronized 使用的锁对象是存储在 Java 对象头里的，jvm 中采用 2 个字来存储对象头(如果对象是数组则会分配3个字，多出来的1个字记录的是数组长度)，其主要结构是由 Mark Word 和 Class Metadata Address 组成

- 如 32 位 JVM 下，除了上述列出的 Mark Word 默认存储结构外，还有如下可能变化的结构：

![image](synchronized_page1.png)

- ObjectMonitor 中有两个队列，_WaitSet和_EntryList, 用来保存 ObjectWaiter 对象列表(每个等待锁的线程都会被封装成 ObjectWaiter 对象)，_owner 指向持有 ObjectMonitor 对象的线程，当多个线程同时访问一段同步代码时，首先会进入_EntryList 集合，当线程获取到对象的 monitor 后进入_Owner 区域并把 monitor 中的 owner 变量设置为当前线程同时 monitor 中的计数器 count 加1，若线程调用 wait() 方法，将释放当前持有的 monitor，owner 变量恢复为 null，count 自减1，同时该线程进入 WaitSet 集合中等待被唤醒。若当前线程执行完毕也将释放monitor(锁)并复位变量的值，以便其他线程进入获取 monitor(锁)。

![image](synchronized_page2.png)

---

### 2.2 synchronized 代码块底层原理
- 同步 demo
```
public class SyncCodeBlock {
   public int i;

   public void syncTask() {
       // 同步代码库
       synchronized (this) {
           i++;
       }
   }
}
```

- 使用 javap 反编译获得字节码
  - 为了保证在方法异常完成时 monitorenter 和 monitorexit 指令依然可以正确配对执行，编译器会自动产生一个异常处理器，这个异常处理器声明可处理所有的异常，它的目的就是用来执行 monitorexit 指令

```
public void syncTask();
    descriptor: ()V
    flags: ACC_PUBLIC
    Code:
      stack=3, locals=3, args_size=1
         0: aload_0
         1: dup
         2: astore_1
         3: monitorenter  //注意此处，进入同步方法
         4: aload_0
         5: dup
         6: getfield      #2             // Field i:I
         9: iconst_1
        10: iadd
        11: putfield      #2            // Field i:I
        14: aload_1
        15: monitorexit   //注意此处，退出同步方法
        16: goto          24
        19: astore_2
        20: aload_1
        21: monitorexit //注意此处，退出同步方法
        22: aload_2
        23: athrow
        24: return
      Exception table:
      // 省略其他字节码.......
```

---

### 2.3 synchronized 方法底层原理
- 方法级的同步是隐式，即无需通过字节码指令来控制的，它实现在方法调用和返回操作之中。JVM 可以从方法常量池中的方法表结构(method_info Structure) 中的 ACC_SYNCHRONIZED 访问标志区分一个方法是否同步方法。

- demo
```
public class SyncMethod {
   public int i;

   public synchronized void syncTask(){
       i++;
   }
}
```

- demo 字节码
```
 public synchronized void syncTask();
    descriptor: ()V
    // 方法标识 ACC_PUBLIC 代表 public 修饰，ACC_SYNCHRONIZED 指明该方法为同步方法
    flags: ACC_PUBLIC, ACC_SYNCHRONIZED
    Code:
      stack=3, locals=1, args_size=1
         0: aload_0
         1: dup
         2: getfield      #2                  // Field i:I
         5: iconst_1
         6: iadd
         7: putfield      #2                  // Field i:I
        10: return
      LineNumberTable:
        line 12: 0
        line 13: 10
```

---

### 2.4 Java 虚拟机对 synchronized 的优化
- 锁的状态总共有四种，无锁状态、偏向锁、轻量级锁和重量级锁
- 锁可以从偏向锁升级到轻量级锁，再升级的重量级锁，但是锁的升级是单向的，也就是说只能从低到高升级，不会出现锁的降级

#### 2.4.1 偏向锁
- 偏向锁是 Java 6 之后加入的新锁，它是一种针对加锁操作的优化手段
- 偏向锁的核心思想是，如果一个线程获得了锁，那么锁就进入偏向模式，此时 Mark Word 的结构也变为偏向锁结构，当这个线程再次请求锁时，无需再做任何同步操作，即获取锁的过程，这样就省去了大量有关锁申请的操作，从而也就提供程序的性能。

#### 2.4.2 轻量级锁
- 倘若偏向锁失败，虚拟机还会尝试使用一种称为轻量级锁的优化手段(1.6之后加入的)，此时 Mark Word 的结构也变为轻量级锁的结构。
- 轻量级锁能够提升程序性能的依据是“对绝大部分的锁，在整个同步周期内都不存在竞争”，注意这是经验数据。
- 轻量级锁所适应的场景是线程交替执行同步块的场合，如果存在同一时间访问同一锁的场合，就会导致轻量级锁膨胀为重量级锁

#### 2.4.3 自旋锁
- 轻量级锁失败后，虚拟机为了避免线程真实地在操作系统层面挂起，还会进行一项称为自旋锁的优化手段。这是基于在大多数情况下，线程持有锁的时间都不会太长，如果直接挂起操作系统层面的线程可能会得不偿失，毕竟操作系统实现线程之间的切换时需要从用户态转换到核心态，因此自旋锁会假设在不久将来，当前的线程可以获得锁，因此虚拟机会让当前想要获取锁的线程做几个空循环(这也是称为自旋的原因)，一般不会太久，可能是50个循环或100循环，在经过若干次循环后，如果得到锁，就顺利进入临界区。

#### 2.4.4 锁消除
- 消除锁是虚拟机另外一种锁的优化，这种优化更彻底，Java 虚拟机在 JIT 编译时(可以简单理解为当某段代码即将第一次被执行时进行编译，又称即时编译)，通过对运行上下文的扫描，去除不可能存在共享资源竞争的锁，通过这种方式消除没有必要的锁，可以节省毫无意义的请求锁时间。

---

## 3. 关于 synchronized 可能需要了解的关键点
### 3. synchronized 的可重入性
- 当一个线程再次请求自己持有对象锁的临界资源时，这种情况属于重入锁，请求将会成功，在 java 中 synchronized 是基于原子性的内部锁机制，是可重入的，因此在一个线程调用 synchronized 方法的同时在其方法体内部调用该对象另一个 synchronized 方法
- 子类也是可以通过可重入锁调用父类的同步方法
- 注意由于 synchronized 是基于 monitor 实现的，因此每次重入，monitor 中的计数器仍会加 1

```
public class AccountingSync implements Runnable{
    static AccountingSync instance = new AccountingSync();
    static int i = 0;
    static int j = 0;
    
    @Override
    public void run() {
        for (int j = 0; j < 1000000; j++) {
            // this, 当前实例对象锁
            synchronized(this) {
                i++;
                increase();  // synchronized 的可重入性
            }
        }
    }

    public synchronized void increase(){
        j++;
    }

    public static void main(String[] args) throws InterruptedException {
        Thread t1 = new Thread(instance);
        Thread t2 = new Thread(instance);
        
        t1.start();
        t2.start();
        
        t1.join();
        t2.join();
        
        System.out.println(i);
    }
}
```

---

### 3.2 线程中断与 synchronized
#### 3.2.1 线程中断
- 在 Java 中，提供了以下3个有关线程中断的方法

```
// 中断线程（实例方法）
public void Thread.interrupt();

// 判断线程是否被中断（实例方法）
public boolean Thread.isInterrupted();

// 判断是否被中断并清除当前中断状态（静态方法）
public static boolean Thread.interrupted();
```

- 当一个线程处于被阻塞状态或者试图执行一个阻塞操作时，使用 Thread.interrupt() 方式中断该线程，注意此时将会抛出一个 InterruptedException 的异常，同时中断状态将会被复位(由中断状态改为非中断状态)

```
public class InterruputSleepThread3 {
    public static void main(String[] args) throws InterruptedException {
        Thread t1 = new Thread() {
            @Override
            public void run() {
                // while 在 try 中，通过异常中断就可以退出 run 循环
                try {
                    while (true) {
                        // 当前线程处于阻塞状态，异常必须捕捉处理，无法往外抛出
                        TimeUnit.SECONDS.sleep(2);
                    }
                } catch (InterruptedException e) {
                    System.out.println("Interruted When Sleep");
                    boolean interrupt = this.isInterrupted();
                    // 中断状态被复位
                    System.out.println("interrupt:"+interrupt);
                }
            }
        };
        
        t1.start();
        TimeUnit.SECONDS.sleep(2);
        
        // 中断处于阻塞状态的线程
        t1.interrupt();

        /**
         * 输出结果:
           Interruted When Sleep
           interrupt:false
         */
    }
}
```

- 除了阻塞中断的情景，还可能会遇到处于运行期且非阻塞的状态的线程，这种情况下，直接调用 Thread.interrupt() 中断线程是不会得到任响应的
- 处于非阻塞状态的线程需要我们手动进行中断检测并结束程序
  - 注意非阻塞状态调用 interrupt() 并不会导致中断状态重置

```
public class InterruputThread {
    public static void main(String[] args) throws InterruptedException {
        Thread t1=new Thread(){
            @Override
            public void run(){
                while(true){
                    // 判断当前线程是否被中断
                    if (this.isInterrupted()){
                        System.out.println("线程中断");
                        break;
                    }
                }

                System.out.println("已跳出循环,线程中断!");
            }
        };
        
        t1.start();
        TimeUnit.SECONDS.sleep(2);
        t1.interrupt();

        /**
         * 输出结果:
            线程中断
            已跳出循环,线程中断!
         */
    }
}
```

#### 3.2.2 中断与 synchronized
- 线程的中断操作对于正在等待获取的锁对象的 synchronized 方法或者代码块并不起作用，也就是对于 synchronized 来说，如果一个线程在等待锁，那么结果只有两种，要么它获得这把锁继续执行，要么它就保存等待，即使调用中断线程的方法，也不会生效

---

### 3.3 等待唤醒机制与 synchronized
- 所谓等待唤醒机制主要指的是 notify/notifyAll 和 wait 方法，在使用这3个方法时，必须处于 synchronized 代码块或者 synchronized 方法中，否则就会抛出 IllegalMonitorStateException 异常，这是因为调用这几个方法前必须拿到当前对象的监视器 monitor 对象，也就是说notify/notifyAll 和 wait 方法依赖于 monitor 对象。
- 需要特别理解的一点是，与 sleep 方法不同的是 wait 方法调用完成后，线程将被暂停，但 wait 方法将会释放当前持有的监视器锁(monitor)，直到有线程调用 notify/notifyAll 方法后方能继续执行，而 sleep 方法只让线程休眠并不释放锁。同时 notify/notifyAll 方法调用后，并不会马上释放监视器锁，而是在相应的 synchronized(){}/synchronized 方法执行结束后才自动释放锁。

---